In [2]:
from ray.rllib.algorithms.ppo import PPOConfig

In [3]:
config = (
    PPOConfig()
    .environment("Taxi-v3")
    .env_runners(num_env_runners=2)
    .framework("torch")
    .training(model={"fcnet_hiddens": [64, 64]})
    .evaluation(evaluation_num_env_runners=1)
)

algo = config.build()

for _ in range(5):
    print(algo.train())

algo.evaluate()

2024-05-24 05:06:40,711	WARNING algorithm_config.py:4077 -- You have specified 1 evaluation workers, but your `evaluation_interval` is 0 or None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
/home/ubuntu/anaconda3/envs/py39/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:525: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/ubuntu/anaconda3/envs/py39/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarn

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.09778279774491826, 'cur_kl_coeff': 0.20000000000000004, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 9.95902478207824, 'policy_loss': 0.011059801381141428, 'vf_loss': 9.947860200943486, 'vf_explained_var': 2.393799443398753e-07, 'kl': 0.000524128453466122, 'entropy': 1.7912276637169622, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 465.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'env_runners': {'episode_reward_max': -632.0, 'episode_reward_min': -929.0, 'episode_reward_mean': -767.9, 'episode_len_mean': 200.0, 'episode_media': {}, 'episodes_this_iter': 20, 'episodes_timesteps_total': 4000, 'policy_reward_min': {}, 'policy_reward_m

{'env_runners': {'episode_reward_max': -362.0,
  'episode_reward_min': -641.0,
  'episode_reward_mean': -508.7,
  'episode_len_mean': 200.0,
  'episode_media': {},
  'episodes_this_iter': 10,
  'episodes_timesteps_total': 2000,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-551.0,
    -452.0,
    -488.0,
    -497.0,
    -641.0,
    -569.0,
    -506.0,
    -362.0,
    -497.0,
    -524.0],
   'episode_lengths': [200, 200, 200, 200, 200, 200, 200, 200, 200, 200]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 0.2196894871126497,
   'mean_inference_ms': 0.7396603614807788,
   'mean_action_processing_ms': 0.08380111503320917,
   'mean_env_wait_ms': 0.04939843100575532,
   'mean_env_render_ms': 0.0},
  'num_faulty_episodes': 0,
  'connector_metrics': {'ObsPreprocessorConnector_ms': 0.014655590057373047,
   'StateBufferConnector_ms': 0.0032830238342285156,
   'ViewRequirementAgentConnector_ms': 

In [4]:
# Another example but I don't know where I got it.

In [4]:
import gymnasium as gym
from ray.rllib.algorithms.ppo import PPOConfig

In [5]:
# Define your problem using python and Farama-Foudation's gymnasium API:

class SimpleCorridor(gym.Env):
    """Corridor in which an agent must learn to move right to reach the exit.
    -----------------------
    | S | 1 | 2 | 3 | G |
    -----------------------
    S = Start, G = goal, corridor_length = 5

    Possible actions to chose from are: 0=left, 1=right
    Observations are floats indicating the current field index, e.g. 0.0 for
    starting position, 1.0 for the field next to the starting position, etc.
    Rewards are -0.1 for all steps, except when reaching the goal (+1.0).
    """

    def __init__(self, config):
        self.end_pos = config["corridor_length"]
        self.cur_pos = 0
        self.action_space = gym.spaces.Discrete(2)
        self.observation_space = gym.spaces.Box(0.0, self.end_pos, shape=(1,))

    def reset(self, *, seed=None, options=None):
        """Resets the episode.
        Returns:
            Initial observation of the new episode and an info dict.
        """

        self.cur_pos = 0
        return [self.cur_pos], {}

    def step(self, action):
        """Takes a single step in the episode given action.

        Returns:
            New observation, reward, terminatd-flag, truncated-flag, info-dict (empty).
        """

        # Walk left
        if action == 0 and self.cur_pos > 0:
            self.cur_pos -= 1
        # Walk right
        elif action == 1:
            self.cur_pos += 1

        # Set terminated flag when endo of corridor (goal) reached.
        terminated = self.cur_pos >= self.end_pos
        truncated = False

        # +1 when goal reached, otherwise -1.
        reward = 1.0 if terminated else -0.1
        return [self.cur_pos], reward, terminated, truncated, {}
        

In [6]:
# Create an RLlib Algorithm instance from a PPOConfig object.

config = (
    PPOConfig().environment(
        # Env class to use (here: our gym.Env sub-class from above).
        env=SimpleCorridor,
        # Config dict to be passed to our custom env's constructor.
        # Use corridor with 20 fields (including S and G).
        env_config = {"corridor_length":28},
    )
    # Parallelize environment rollouts.
    .env_runners(num_env_runners=3)
)

# Construct the actual (PPO) algorithm object from the config.
algo = config.build()

/home/ubuntu/anaconda3/envs/py39/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:525: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/ubuntu/anaconda3/envs/py39/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/ubuntu/anaconda3/envs/py39/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is depreca

In [7]:
# Train for n iterations and report results (mean, episode rewards).
# Since we have to move at least 19 times in the env to reach the goal and
# each move gives us -0.1 reward (except the last move at the end: +1.0),
# we can expect to reach an optimal episode reward of -0.1*18 + 1.0 = -0.8

for i in range(10):
    results = algo.train()
    print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}")

# Perform inference (action computations) based on given env observations.
# Note that we are using a slightly different env here (len 10 instead of 20),
# however, this should still work as the agent has (hopefully) learned
# to "just always walk right!"

env = SimpleCorridor({"corridor_length": 10})

# Get the initial observation (should be: [0.0] for the starting position).
obs, info = env.reset()
terminated = truncated = False
total_reward = 0.0

# Play one episode.
while not terminated and not truncated:
    # Compute a single action, given the current observation
    # from the environment.
    action = algo.compute_single_action(obs)
    # Apply the computed action in the environment.
    obs, reward, terminated, truncated, info = env.step(action)
    # Sum up rewards for reporting purposes.
    total_reward += reward

# Report results.
print(f"Played 1 episode; total-reward={total_reward}")


KeyError: 'episode_reward_mean'